In [ ]:
# Exercise 1

import pandas as pd

data = pd.read_csv('garments_worker_productivity.csv')

# Drop the date column
if 'date' in data.columns:
    data.drop(columns=['date'], inplace=True)

# For each of the categorical attributes, print all the unique elements.
categorical_columns = ['day', 'quarter', 'department', 'team']
unique_values = {col: data[col].unique() for col in categorical_columns}
print(unique_values)
# From the unique values output, we can see that the there is a duplicated values in department column, which is 'finishing'
# since one of them has extra space after the word.
# We need to clean the data

# Data cleaning for categorical attributes
data['day'] = data['day'].str.strip().str.lower()
data['quarter'] = data['quarter'].str.strip().str.lower()
data['department'] = data['department'].str.strip().str.lower()

unique_values = {col: data[col].unique() for col in categorical_columns}

# Create another column named satisfied that records the productivity performance.
data['satisfied'] = (data['actual_productivity'] >= data['targeted_productivity']).astype(int)

# Drop the columns actual_productivity and targeted_productivity
data.drop(columns=['actual_productivity', 'targeted_productivity'], inplace=True)

# Find and print which columns/attributes have empty values
missing_columns = data.columns[data.isnull().any()]
print(missing_columns) # wip has empty value

# Fill the empty values with 0
data.fillna(0, inplace=True)

{'day': array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object), 'quarter': array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
      dtype=object), 'department': array(['sweing', 'finishing ', 'finishing'], dtype=object), 'team': array([ 8,  1, 11, 12,  6,  7,  2,  3,  9, 10,  5,  4])}
Index(['wip'], dtype='object')
Index([], dtype='object')


In [27]:
# Exercise 2

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report

data = pd.read_csv('garments_worker_productivity.csv')

# Drop the date column
if 'date' in data.columns:
    data.drop(columns=['date'], inplace=True)

# For each of the categorical attributes, print all the unique elements.
categorical_columns = ['day', 'quarter', 'department', 'team']
unique_values = {col: data[col].unique() for col in categorical_columns}

# Data cleaning for categorical attributes
data['day'] = data['day'].str.strip().str.lower()
data['quarter'] = data['quarter'].str.strip().str.lower()
data['department'] = data['department'].str.strip().str.lower()

# Create another column named satisfied that records the productivity performance.
data['satisfied'] = (data['actual_productivity'] >= data['targeted_productivity']).astype(int)

# Drop the columns actual_productivity and targeted_productivity
data.drop(columns=['actual_productivity', 'targeted_productivity'], inplace=True)

# Find and print which columns/attributes have empty values
missing_columns = data.columns[data.isnull().any()]

# Fill the empty values with 0
data.fillna(0, inplace=True)


# Exercise 2.1
# For each of the categorical attributes, encode the set of categories using integers 0, ... n - 1
label_encoders = {}
for col in ['day', 'quarter', 'department', 'team']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Split the data into training and testing sets with a ratio of 80:20
X = data.drop(columns=['satisfied'])
y = data['satisfied']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Exercise 2.2
# Use the categorical attributes only
categorical_features = ['day', 'quarter', 'department', 'team']

# please build a Categorical Naïve Bayes classifier that predicts the column satisfied. 
X_train_cat = X_train[categorical_features]
X_test_cat = X_test[categorical_features]

nb_model = CategoricalNB()
nb_model.fit(X_train_cat, y_train)

y_pred = nb_model.predict(X_test_cat)

# Report the testing result using classification_report
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
0,0.434783,0.158730,0.232558,63.000
1,0.755760,0.926554,0.832487,177.000
accuracy,0.725000,0.725000,0.725000,0.725
macro avg,0.595271,0.542642,0.532523,240.000
weighted avg,0.671504,0.725000,0.675006,240.000
